# Evaluate Similarity Grouping

In this notebook, we evaluate how effective a relation can be integrated using the NoiseAwareGroupBy Operator.
Therefore, we utilize the [Music Brainz 20K](https://dbs.uni-leipzig.de/research/projects/benchmark-datasets-for-entity-resolution).

The dataset contains modified (usign the DAPO data generator) song records from different sources.
The goal is to group same songs into buckets. E.g. The records {"title": "Daniel Balavoine - L'enfant aux yeux d'Italie", "artist": null, "album": "De vous à elle en passant par moi", ...} and {"name": L'enfant aux yeux d'Italie - De vous à elle en passant par moi", "artist": "Daniel Balavoine", "album": null} describe the same song.

The column "CID" describes the cluster of the record. Using the  `SoftAggregateScikit` operator, we determine clusters and calculate the metrics:
* Adjusted Rand Index (ARI)
* Normalized Mutual Information (NMI)
* Fowlkes-Mallows Index (FMI)


In [1]:
%%capture
!pip3 install faiss-gpu-cu12
!pip3 install pgvector

In [1]:
%%capture
!rm -rf SofteningQueryEvaluation
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd
import time

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel

from db.operators import Dummy, SoftAggregateScikit
from db.operators.Aggregate import SetAggregation
from sklearn.cluster import KMeans, DBSCAN, HDBSCAN

from sklearn.metrics import rand_score, adjusted_rand_score
from sklearn.metrics import fowlkes_mallows_score
from sklearn.metrics import mutual_info_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.metrics import homogeneity_score, completeness_score
from sklearn.metrics import v_measure_score, homogeneity_completeness_v_measure, homogeneity_completeness_v_measure

In [4]:
m = ModelMgr()
stem = SentenceTransformerEmbeddingModel(m)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [7]:
df_music = pd.read_csv("musicbrainz-20-A01.csv", index_col=0).drop(columns=["length"], axis=1)
significant_cols = ["title", "artist", "album", "year", "language"]
df_music.head()

,CID,CTID,SourceID,id,number,title,artist,album,year,language
TID,,,,,,,,,,
1,1,1,2,MBox7368722-HH,9,Daniel Balavoine - L'enfant aux yeux d'Italie,NaN,De vous à elle en passant par moi,75,French
2,2512,5,4,139137-A047,7,007,[unknown],Cantigas de roda (unknown),NaN,Por.
3,2,1,2,MBox38440522-HH,17,Action PAINTING! - Mustard Gas,NaN,There and Back Again Lane,95,English
4,3,1,5,4489993,10,Your Grace,Kathy Troccoli,Comfort,2005,English
5,4,1,5,10339621,2,Well You Needn't,Ernie Stadler Jazz Quintet,First Down,2010,English


In [70]:
def evaluate(df, cluster_columns, id_column, cluster_class, cluster_params, serialization_mode, reduce_dimensions, drop_na):
    key = (str(cluster_class), str(serialization_mode), str(reduce_dimensions), str(drop_na))

    if drop_na:
        df = df.dropna()

    if cluster_class == KMeans:
        cluster_params = {"n_clusters": len(df["CID"].unique())}

    columns = [col.strip() for col in df.columns]
    data = [[str(y) for y in x] for x in df.itertuples(name=None)]

    d = Dummy("data", ["tid"] + columns, data).open()
    agg = SoftAggregateScikit(
        d,
        cluster_columns,
        [SetAggregation("tid", "ids")],
        em=stem,
        cluster_class = cluster_class,
        cluster_params = cluster_params,
        serialization_mode = serialization_mode,
        reduce_dimensions = reduce_dimensions
    )

    tic = time.time()
    result = agg.open().fetch_all()
    toc = time.time()


    predictions = []
    for i, row in enumerate(result):
        predictions.append(pd.Series([i for _ in range(len(row["ids"]))], index=[int(idx) for idx in row["ids"]]))

    predicted_labels = pd.concat(predictions).sort_index()
    true_labels = df[id_column].sort_index()

    return key, {
        "rand_score": rand_score(true_labels, predicted_labels),
        "adjusted_rand_score": adjusted_rand_score(true_labels, predicted_labels),
        "fowlkes_mallows_score": fowlkes_mallows_score(true_labels, predicted_labels),
        "mutual_info_score": mutual_info_score(true_labels, predicted_labels),
        "adjusted_mutual_info_score": adjusted_mutual_info_score(true_labels, predicted_labels),
        "normalized_mutual_info_score": normalized_mutual_info_score(true_labels, predicted_labels),
        "homogeneity_score": homogeneity_score(true_labels, predicted_labels),
        "completeness_score": completeness_score(true_labels, predicted_labels),
        "v_measure_score": v_measure_score(true_labels, predicted_labels),
        "homogeneity_completeness_v_measure": homogeneity_completeness_v_measure(true_labels, predicted_labels),
        "runtime": toc-tic,
        "pred": predicted_labels
    }

In [71]:
overall_results = {}

cluster_classes = [
    (KMeans, None),
    (DBSCAN, {"eps": 0.1, "min_samples": 1}),
    (HDBSCAN, {"min_cluster_size": 2}),
]

for cc, cp in cluster_classes:
    for sm in ["FIELD_SERIALIZED", "FULL_SERIALIZED"]:
        for dn in [True, False]:
            for dim in [None, 2, 10, 50, 100]:
                res = evaluate(df_music, significant_cols, "CID", cluster_class=cc, cluster_params=cp, serialization_mode = sm, reduce_dimensions = dim, drop_na = dn)
                overall_results[res[0]] = res[1]
                print(res[0], res[1]["adjusted_rand_score"])

("<class 'sklearn.cluster._kmeans.KMeans'>", 'FIELD_SERIALIZED', 'None', 'True') 0.7227659245977957


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._kmeans.KMeans'>", 'FIELD_SERIALIZED', '2', 'True') -2.4531393179505998e-05


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._kmeans.KMeans'>", 'FIELD_SERIALIZED', '10', 'True') 0.008484851851779879


...

("<class 'sklearn.cluster._hdbscan.hdbscan.HDBSCAN'>", 'FULL_SERIALIZED', '10', 'False') 0.5092503647132791


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._hdbscan.hdbscan.HDBSCAN'>", 'FULL_SERIALIZED', '50', 'False') 0.511497910753623


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._hdbscan.hdbscan.HDBSCAN'>", 'FULL_SERIALIZED', '100', 'False') 0.5013349419763493


In [72]:
keys = ["cluster", "serialization", "dimension", "drop_na"]
evaluation_results_list = [v | {ki: vi for ki, vi in zip(keys, k)} for k, v in overall_results.items()]
df_evaluation_results = pd.DataFrame.from_records(evaluation_results_list)
df_evaluation_results["cluster"] = df_evaluation_results["cluster"].apply(lambda x: x.split(".")[-1].replace("'>", ""))
df_evaluation_results = df_evaluation_results.set_index(keys)
df_evaluation_results.head()

rand_score  adjusted_rand_score  \
cluster serialization    dimension drop_na                                    
KMeans  FIELD_SERIALIZED None      True       0.999987             0.722766   
                         2         True       0.999951            -0.000025   
                         10        True       0.999947             0.008485   
                         50        True       0.999949             0.017519   
                         100       True       0.999944             0.007973   

                                            fowlkes_mallows_score  \
cluster serialization    dimension drop_na                          
KMeans  FIELD_SERIALIZED None      True                  0.722914   
                         2         True                  0.000000   
                         10        True                  0.008618   
                         50        True                  0.017698   
                         100       True                  0.008179   

                                            mutual_info_score  \
cluster serialization    dimension drop_na                      
KMeans  FIELD_SERIALIZED None      True              7.937553   
                         2         True              7.901016   
                         10        True              7.899162   
                         50        True              7.900311   
                         100       True              7.897666   

                                            adjusted_mutual_info_score  \
cluster serialization    dimension drop_na                               
KMeans  FIELD_SERIALIZED None      True                       0.736875   
                         2         True                      -0.000026   
                         10        True                       0.009570   
                         50        True                       0.019306   
                         100       True                       0.009422   

                                            normalized_mutual_info_score  \
cluster serialization    dimension drop_na                                 
KMeans  FIELD_SERIALIZED None      True                         0.998469   
                         2         True                         0.994026   
                         10        True                         0.993938   
                         50        True                         0.994040   
                         100       True                         0.993843   

                                            homogeneity_score  \
cluster serialization    dimension drop_na                      
KMeans  FIELD_SERIALIZED None      True              0.998429   
                         2         True              0.993833   
                         10        True              0.993600   
                         50        True              0.993744   
                         100       True              0.993411   

                                            completeness_score  \
cluster serialization    dimension drop_na                       
KMeans  FIELD_SERIALIZED None      True               0.998509   
                         2         True               0.994220   
                         10        True               0.994277   
                         50        True               0.994336   
                         100       True               0.994276   

                                            v_measure_score  \
cluster serialization    dimension drop_na                    
KMeans  FIELD_SERIALIZED None      True            0.998469   
                         2         True            0.994026   
                         10        True            0.993938   
                         50        True            0.994040   
                         100       True            0.993843   

                                                           homogeneity_completeness_v_measure  \
cluster serialization    dimension drop_

In [81]:
df_evaluation_results.to_pickle("EvaluateClustering.pkl")
df_evaluation_results.drop(columns=["pred"]).to_csv("EvaluateClustering.csv")

In [80]:
for x,y in df_evaluation_results.iterrows():
  print(y["pred"])
  break

4           0
5           1
9           2
10          3
19          4
         ... 
19296    2867
19302    2868
19319      83
19332    2869
19341    2870
Length: 2969, dtype: int64
